In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

In [3]:
main_user = pd.read_csv("user_segment.csv")

main_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160582 entries, 0 to 160581
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   treatment          160582 non-null  object 
 1   country            160527 non-null  object 
 2   mobile_source      160582 non-null  object 
 3   device_ram         160582 non-null  int64  
 4   dpi                160582 non-null  int64  
 5   days_complete_ch1  160582 non-null  int64  
 6   sessions_ch1       160582 non-null  int64  
 7   purchases_ch1      160582 non-null  int64  
 8   revenue_ch1        160582 non-null  float64
 9   coins_bought_ch1   160582 non-null  int64  
 10  cash_bought_ch1    160582 non-null  int64  
 11  median_coins_ch1   156501 non-null  float64
 12  median_cash_ch1    156501 non-null  float64
 13  conversion_ch1     160582 non-null  int64  
 14  revenue            160582 non-null  float64
 15  purchases          160582 non-null  int64  
 16  co

In [4]:
#Users to be seprated according to treatment 
user_no_promo = main_user[(main_user["treatment"]== 'no_promo')]

#user_no_promo.info()

In [5]:
user_0days = main_user[(main_user["treatment"]== 'after0days')]

#user_0days.info()

In [6]:
user_25days = main_user[(main_user["treatment"]== 'after25days')]

#user_25days.info()

In [7]:
user_50days = main_user[(main_user["treatment"]== 'after50days')]

#user_50days.info()

In [8]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(main_user, main_user["treatment"]):
    train_set = main_user.loc[train_index]
    test_set = main_user.loc[test_index]


In [177]:
test_set["treatment"].value_counts() / len(test_set)

after0days     0.341782
after25days    0.327895
after50days    0.165177
no_promo       0.165146
Name: treatment, dtype: float64

In [178]:
train_set["treatment"].value_counts() / len(train_set)

after0days     0.341774
after25days    0.327879
after50days    0.165189
no_promo       0.165158
Name: treatment, dtype: float64

In [179]:
main_user["treatment"].value_counts() / len(main_user)

after0days     0.341776
after25days    0.327882
after50days    0.165187
no_promo       0.165155
Name: treatment, dtype: float64

In [9]:
user_profile = main_user.drop(["treatment", "country"], axis = 1)

user_profile = pd.get_dummies(user_profile, columns=["mobile_source"], prefix = ["source"])

user_profile = user_profile.reset_index()

user_profile = user_profile.astype(float)

scaler = StandardScaler()

user_scaled = user_profile[np.isfinite(user_profile).all(1)]

user_scaled = pd.DataFrame(user_scaled)

user_scaled.info()

user_scaled.to_csv('user_scaled.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156501 entries, 0 to 160581
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   index                 156501 non-null  float64
 1   device_ram            156501 non-null  float64
 2   dpi                   156501 non-null  float64
 3   days_complete_ch1     156501 non-null  float64
 4   sessions_ch1          156501 non-null  float64
 5   purchases_ch1         156501 non-null  float64
 6   revenue_ch1           156501 non-null  float64
 7   coins_bought_ch1      156501 non-null  float64
 8   cash_bought_ch1       156501 non-null  float64
 9   median_coins_ch1      156501 non-null  float64
 10  median_cash_ch1       156501 non-null  float64
 11  conversion_ch1        156501 non-null  float64
 12  revenue               156501 non-null  float64
 13  purchases             156501 non-null  float64
 14  conversion            156501 non-null  float64
 15  

In [10]:
poly = PolynomialFeatures(interaction_only=True,include_bias = False)
regression = LinearRegression(normalize=True)

interactions = list()
x  = user_scaled.drop(['revenue','conversion', 'purchases','index', 'treatment2'], axis = 1)

y = user_scaled[['revenue']]

z = poly.fit_transform(x)

z = pd.DataFrame(z)

z_1 = poly.get_feature_names_out(input_features=None)

z.columns = z_1

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, x, y, cv=crossvalidation, n_jobs=1))

for features in z.columns:
    test_col = z[[features]]
    score = np.mean(cross_val_score(regression, test_col, y, cv=crossvalidation, n_jobs=1))
    interactions.append((features, round(score,3)))
    
top_10_inter = sorted(interactions, key=lambda x:x[1], reverse=True)[:10]

top_10_inter

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

...

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipe

[('cash_bought_ch1', 0.073),
 ('cash_bought_ch1 conversion_ch1', 0.073),
 ('revenue_ch1', 0.069),
 ('revenue_ch1 conversion_ch1', 0.069),
 ('cash_bought_ch1 median_coins_ch1', 0.062),
 ('device_ram cash_bought_ch1', 0.061),
 ('cash_bought_ch1 us', 0.058),
 ('revenue_ch1 median_coins_ch1', 0.056),
 ('device_ram revenue_ch1', 0.055),
 ('dpi cash_bought_ch1', 0.055)]

In [16]:
inter_df = pd.DataFrame()

inter_name = [x[0] for x in interactions]
inter_score = [x[1] for x in interactions]

inter_df['feature_name'] = inter_name
inter_df['feature_score'] = inter_score

l_1 = ['cash_bought_ch1 median_coins_ch1',
       'revenue_ch1 conversion_ch1',
       'revenue_ch1 median_coins_ch1',
       'device_ram revenue_ch1',
       'device_ram cash_bought_ch1',
       'dpi cash_bought_ch1',
       'cash_bought_ch1 conversion_ch1',
       'cash_bought_ch1 us',
       'cash_bought_ch1 t25',
       'conversion_ch1 t25',
       'revenue_ch1 t25']


inter_df = z[[c for c in z.columns if c in l_1]]

inter_df.to_excel('interact_values')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156501 entries, 0 to 156500
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   device_ram revenue_ch1            156501 non-null  float64
 1   device_ram cash_bought_ch1        156501 non-null  float64
 2   dpi cash_bought_ch1               156501 non-null  float64
 3   revenue_ch1 median_coins_ch1      156501 non-null  float64
 4   revenue_ch1 conversion_ch1        156501 non-null  float64
 5   revenue_ch1 t25                   156501 non-null  float64
 6   cash_bought_ch1 median_coins_ch1  156501 non-null  float64
 7   cash_bought_ch1 conversion_ch1    156501 non-null  float64
 8   cash_bought_ch1 t25               156501 non-null  float64
 9   cash_bought_ch1 us                156501 non-null  float64
 10  conversion_ch1 t25                156501 non-null  float64
dtypes: float64(11)
memory usage: 13.1 MB


In [1]:
inter_df.to_excel('interact_values.xlsx')

NameError: name 'inter_df' is not defined

In [4]:
user_final = pd.read_csv('user_interact_final.csv')

scaler = StandardScaler()

user_final[['device_ram','dpi','days_complete_ch1', 'sessions_ch1','purchases_ch1',
             'revenue_ch1', 'coins_bought_ch1', 'cash_bought_ch1','median_coins_ch1',
              'median_coins_ch1', 'median_cash_ch1', 'revenue', 'purchases',
             'ad_revenue']] = scaler.fit_transform(user_final[['device_ram','dpi','days_complete_ch1', 'sessions_ch1','purchases_ch1',
             'revenue_ch1', 'coins_bought_ch1', 'cash_bought_ch1','median_coins_ch1',
              'median_coins_ch1', 'median_cash_ch1', 'revenue', 'purchases',
             'ad_revenue']])

In [22]:
k1 = []
inertia_s1 = []

for i in tqdm(range(2,20)):
    k1.append(i)
    kmeans1 = KMeans(n_clusters=i,random_state=125,max_iter=100).fit(user_final)
    inertia_s1.append(kmeans1.inertia_)
    


100%|██████████| 18/18 [03:33<00:00, 11.89s/it]


In [23]:
plt.figure(figsize=(15,5))
plt.plot(k1,inertia_s1,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=5)
plt.title('Inertia (SSE) vs. K Value')
plt.xlabel('K')
plt.ylabel('Inertia score (SSE)')


Text(0, 0.5, 'Inertia score (SSE)')

In [33]:
kmeans = KMeans(n_clusters=5,random_state=125,max_iter=100).fit(user_final)
sscore = silhouette_score(user_final,kmeans.labels_)
inertia= kmeans.inertia_

# save k-means
#pkl_filename = "./files/model_objects/kmeans_model.pkl"
#with open(pkl_filename, 'wb') as file:
#    pickle.dump(kmeans, file)



In [7]:
#df_features = pd.DataFrame(columns = user_scaled.columns)
user_final['cluster_ids'] = kmeans.labels_

In [17]:

# Overall level summary
df_profile_overall = user_final.describe().T

# using mean; use appropriate summarization (median, count, etc.) for each feature
df_profile_overall['Overall Dataset'] = df_profile_overall[['mean']]
df_profile_overall = df_profile_overall[['Overall Dataset']]

# Cluster ID level summary
df_cluster_summary_m = user_final.groupby('cluster_ids').describe().T.reset_index()
#df_cluster_summay_m = df_cluster_summary.rename(columns={'level_0':'column','level_1':'metric'})

# using mean; use appropriate summarization (median, count, etc.) for each feature
#df_cluster_summary = df_cluster_summary_m[df_cluster_summary_m['metric'] == "mean"]
#df_cluster_summary = df_cluster_summary.set_index('column')

#df_cluster_count = df_cluster_summary_m[df_cluster_summary_m['metric'] == "count"]
#df_cluster_count = df_cluster_count.set_index('column')
# join into single summary dataset
#df_profile_1 = df_cluster_summary.join(df_profile_overall) # joins on Index
#df_profile_2 = df_cluster_count.join(df_profile_overall) # joins on Index

my_list = list(df_cluster_summary_m)
my_list

['level_0', 'level_1', 0, 1, 2, 3, 4]

In [10]:
df_cluster_summay_m = user_final.groupby('cluster_ids').describe().T.reset_index()
df_cluster_summay_m = df_cluster_summary_m.rename(columns={'level_0':'column','level_1':'metric'}, inplace = 'True')
#df_cluster_summary_m

df_cluster_summary = df_cluster_summary_m[df_cluster_summary_m['metric'] == "mean"]
df_cluster_summary = df_cluster_summary.set_index('column').T
#df_cluster_summary

df_cluster_count = df_cluster_summary_m[df_cluster_summary_m['metric'] == "count"]
df_cluster_count = df_cluster_count.set_index('column').T

my_list = list(df_cluster_summary)
df_cluster_count = df_cluster_count[[
        'conversion',
        'high_engage',
        'high_value',
        't0',
        't25',
        't50',
         'c',
         'us',
         'uk',
         'de',
         'fr',
         'source_Featuring',
         'source_Organic',
         'source_Paid',
         'source_Wooga Network']]

In [20]:
# Cluster ID level summary
df_cluster_summary_m.to_excel("scaled_segments.xlsx")

In [12]:
df_cluster_mean = df_cluster_summary[['device_ram',
                                         'dpi',
                                         'days_complete_ch1',
                                         'sessions_ch1',
                                         'purchases_ch1',
                                         'revenue_ch1',
                                         'coins_bought_ch1',
                                         'cash_bought_ch1',
                                         'median_coins_ch1',
                                         'median_cash_ch1',
                                         'revenue',
                                         'purchases',
                                         'ad_revenue']]

df_cluster_mean.T

cluster_ids       metric         0          1         2          3         4
column                                                                      
device_ram          mean  0.006146   0.496385  0.009247   0.352842 -0.018173
dpi                 mean  0.005128  -0.089326  0.000572   0.121593 -0.006616
days_complete_ch1   mean  0.014492  -0.137058  0.029904  -0.150313 -0.043003
sessions_ch1        mean  0.056691  -0.005456  0.006423    0.09576 -0.063813
purchases_ch1       mean -0.035888  13.706111 -0.023391  11.807255 -0.035283
revenue_ch1         mean -0.038885  62.356319 -0.035541  12.880794  -0.03971
coins_bought_ch1    mean -0.006087   1.877023 -0.011674    3.60394 -0.010602
cash_bought_ch1     mean -0.037372  74.379775 -0.036482  12.113582 -0.037126
median_coins_ch1    mean  0.019608   1.272346  0.042397   0.538298 -0.066176
median_cash_ch1     mean -0.047965    0.37364  0.010675  -0.203566   0.03885
revenue             mean -0.011702  17.196097 -0.019105   3.942167 -0.003745
purchases           mean -0.009154   4.107945 -0.018457   3.827861 -0.003041
ad_revenue          mean  0.028798  -0.306105  -0.04093  -0.289104  0.014163

In [13]:
df_cluster_summary.T

cluster_ids                      metric             0               1  \
column                                                                  
F1                                 mean  79961.069516    65690.666667   
index                              mean  79961.069516    65690.666667   
device_ram                         mean      0.006146        0.496385   
dpi                                mean      0.005128       -0.089326   
days_complete_ch1                  mean      0.014492       -0.137058   
sessions_ch1                       mean      0.056691       -0.005456   
purchases_ch1                      mean     -0.035888       13.706111   
revenue_ch1                        mean     -0.038885       62.356319   
coins_bought_ch1                   mean     -0.006087        1.877023   
cash_bought_ch1                    mean     -0.037372       74.379775   
median_coins_ch1                   mean      0.019608        1.272346   
median_cash_ch1                    mean     -0.047965         0.37364   
conversion_ch1                     mean      0.002761             1.0   
revenue                            mean     -0.011702       17.196097   
purchases                          mean     -0.009154        4.107945   
conversion                         mean       0.05546             1.0   
ad_revenue                         mean      0.028798       -0.306105   
treatment2                         mean      2.152342             2.0   
high_engage                        mean      0.490143        0.333333   
high_value                         mean      0.505523            0.75   
t0                                 mean      0.343171        0.416667   
t25                                mean      0.326602        0.333333   
t50                                mean      0.164941        0.083333   
c                                  mean      0.165286        0.166667   
us                                 mean      0.326679        0.916667   
uk                                 mean      0.059966             0.0   
de                                 mean      0.143424             0.0   
fr                                 mean      0.113796             0.0   
source_Featuring                   mean       0.01398        0.166667   
source_Organic                     mean      0.404863        0.166667   
source_Paid                        mean      0.573448        0.666667   
source_Wooga Network               mean      0.007709             0.0   
device_ram revenue_ch1             mean     33.590185   187847.495793   
device_ram cash_bought_ch1         mean    244.724428  2218703.333333   
dpi cash_bought_ch1                mean     35.976585   282658.333333   
revenue_ch1 median_coins_ch1       mean     59.469774   493588.824342   
revenue_ch1 conversion_ch1         mean      0.016216       83.561518   
revenue_ch1 t25                    mean      0.003864       22.804002   
cash_bought_ch1 median_coins_ch1   mean     406.31688  5635080.833333   
cash_bought_ch1 conversion_ch1     mean      0.118226      986.666667   
cash_bought_ch1 t25                mean      0.031354           237.5   
cash_bought_ch1 us                 mean      0.057531      932.916667   
conversion_ch1 t25                 mean      0.000748        0.333333   

cluster_ids                                  2              3              4  
column                                                                        
F1                                26499.761796    75086.64268  133701.066853  
index                             26499.761796    75086.64268  133701.066853  
device_ram                            0.009247       0.352842      -0.018173  
dpi                                   0.000572       0.121593      -0.006616  
days_complete_ch1                     0.029904      -0.150313      -0.043003  
sessions_ch1                          0.006423        0.09576      -0.063813  
purchases_ch1                        -0.023391      11.807255      -0.035283  
rev

In [14]:
df_cluster_summay_m.to_excel('scaled_segments')

AttributeError: 'NoneType' object has no attribute 'to_excel'

[('log_revenue', 0.304),
 ('median_coins_ch1 log_revenue', 0.277),
 ('index log_revenue', 0.252),
 ('device_ram log_revenue', 0.25),
 ('treatment2 log_revenue', 0.247),
 ('dpi log_revenue', 0.245),
 ('log_revenue us', 0.215),
 ('log_revenue source_Paid', 0.211),
 ('high_engage log_revenue', 0.193),
 ('median_cash_ch1 log_revenue', 0.188)]

In [24]:
#Vizuals
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
import numpy as np

pca = PCA(2)
df_viz = pca.fit_transform(user_final)

In [26]:
label = kmeans.fit_predict(df_viz)

In [37]:
u_labels = np.unique(label)
centers = np.array(kmeans.cluster_centers_)

In [1]:

plt.scatter(centers[:,0], centers[:,1], marker="x", color='r')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [15]:
viz_data = pd.read_excel('scaled_segments.xlsx')


a = viz_data[viz_data['level_1'] == "mean"]

In [28]:
#viz_data.drop(['Unnamed: 0'], axis = 1, inplace = True)

a = viz_data[viz_data['level_1'] == "mean"]

In [33]:
a

column metric             0             1  \
1                                  F1   mean  79961.069516  6.569067e+04   
9                               index   mean  79961.069516  6.569067e+04   
17                         device_ram   mean      0.006146  4.963847e-01   
25                                dpi   mean      0.005128 -8.932648e-02   
33                  days_complete_ch1   mean      0.014492 -1.370580e-01   
41                       sessions_ch1   mean      0.056691 -5.456211e-03   
49                      purchases_ch1   mean     -0.035888  1.370611e+01   
57                        revenue_ch1   mean     -0.038885  6.235632e+01   
65                   coins_bought_ch1   mean     -0.006087  1.877023e+00   
73                    cash_bought_ch1   mean     -0.037372  7.437978e+01   
81                   median_coins_ch1   mean      0.019608  1.272346e+00   
89                    median_cash_ch1   mean     -0.047965  3.736397e-01   
97                     conversion_ch1   mean      0.002761  1.000000e+00   
105                           revenue   mean     -0.011702  1.719610e+01   
113                         purchases   mean     -0.009154  4.107945e+00   
121                        conversion   mean      0.055460  1.000000e+00   
129                        ad_revenue   mean      0.028798 -3.061055e-01   
137                        treatment2   mean      2.152342  2.000000e+00   
145                       high_engage   mean      0.490143  3.333333e-01   
153                        high_value   mean      0.505523  7.500000e-01   
161                                t0   mean      0.343171  4.166667e-01   
169                               t25   mean      0.326602  3.333333e-01   
177                               t50   mean      0.164941  8.333333e-02   
185                                 c   mean      0.165286  1.666667e-01   
193                                us   mean      0.326679  9.166667e-01   
201                                uk   mean      0.059966  0.000000e+00   
209                                de   mean      0.143424  0.000000e+00   
217                                fr   mean      0.113796  0.000000e+00   
225                  source_Featuring   mean      0.013980  1.666667e-01   
233                    source_Organic   mean      0.404863  1.666667e-01   
241                       source_Paid   mean      0.573448  6.666667e-01   
249              source_Wooga Network   mean      0.007709  0.000000e+00   
257            device_ram revenue_ch1   mean     33.590185  1.878475e+05   
265        device_ram cash_bought_ch1   mean    244.724428  2.218703e+06   
273               dpi cash_bought_ch1   mean     35.976585  2.826583e+05   
281      revenue_ch1 median_coins_ch1   mean     59.469774  4.935888e+05   
289        revenue_ch1 conversion_ch1   mean      0.016216  8.356152e+01   
297                   revenue_ch1 t25   mean      0.003864  2.280400e+01   
305  cash_bought_ch1 median_coins_ch1   mean    406.316880  5.635081e+06   
313    cash_bought_ch1 conversion_ch1   mean      0.118226  9.866667e+02   
321               cash_bought_ch1 t25   mean      0.031354  2.375000e+02   
329                cash_bought_ch1 us   mean      0.057531  9.329167e+02   
337                conversion_ch1 t25   mean      0.000748  3.333333e-01   

                2              3              4  
1    26499.761796   75086.642680  133701.066853  
9    26499.761796   75086.642680  133701.066853  
17       0.009247       0.352842      -0.018173  
25       0.000572       0.121593      -0.006616  
33       0.029904      -0.150313      -0.043003  
41       0.006423       0.095760      -0.063813  
49      -0.023391      11.807255      -0.035283  
57      -0.035541      12.880794      -0.039710  
65      -0.011674       3.603940      -0.010602  
73      -0.036482      12.113582      -0.037126  
81       0.042397       0.538298      -0.066176  
89       0.010675      -0.203566       0.038850  
97       0.004247       1.000000    